In [1]:
import pandas as pd 
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Embedding, LSTM

In [2]:
df_training = pd.read_csv('data_training.csv')
df_test = pd.read_csv('data_test.csv')

In [3]:
df_training.drop(columns=['origem', 'arquivo'], inplace=True)
df_test.drop(columns=['origem', 'arquivo'], inplace=True)

In [4]:
df_training.rename(columns={'topico':'label', 'texto':'category'}, inplace=True)
df_test.rename(columns={'topico':'label', 'texto':'category'}, inplace=True)

In [5]:
df_training.head()

,label,category
0,coconut-oil,us scientists say tropical oils health risk au...
1,coconut-oil,vegetable oils may tighten despite seed surplu...
2,coconut-oil,corrected philippines criticises ec for oil le...
3,coconut-oil,coconut oil contract to change dutch traders r...
4,alum,feb daily ave unwrought aluminium output tonne...


In [6]:
df_test.head()

,label,category
0,coconut-oil,us tropical oil labeling plan suffers defeat w...
1,coconut-oil,growth of palm oil use set to slow output to r...
2,coconut-oil,philippine coconut chief to lobby against ec t...
3,alum,alcoa raising prices of aluminum beverage can ...
4,alum,indonesia raises stake in aluminium plant jaka...


In [64]:
x_train = df_training['category'].tolist()
y_train = df_training['label'].tolist()

x_test = df_test['category'].tolist()
y_test = df_test['label'].tolist()

In [106]:
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))

11413
11413
4024
4024


tokenizer: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [65]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
train_sequences= tokenizer.texts_to_sequences(x_train)

train_padded = pad_sequences(train_sequences, maxlen=200, padding='post', truncating='post')

In [84]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_test)
test_sequences= tokenizer.texts_to_sequences(x_test)

test_padded = pad_sequences(test_sequences, maxlen=200, padding='post', truncating='post')

In [110]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(y_train)
label_word_index = label_tokenizer.word_index
label_sequences = tokenizer.texts_to_sequences(y_train)

training_label_seq = np.asarray(label_sequences)

/tmp/ipykernel_26597/1506415749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_label_seq = np.asarray(label_sequences)


In [ ]:
print(dict(list(label_word_index.items())))
len(label_word_index)


In [104]:
print(label_sequences)

[[683, 42], [683, 42], [683, 42], [683, 42], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [3021], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [182], [1

In [12]:
#reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

#def decode_article(text):
#    return ' '.join([reverse_word_index.get(i, '?') for i in text])
#print(decode_article(train_padded[10]))
#print('---')
#print(x_train[10])

In [13]:
vocab_size = 5000 #train_set_X.shape[1]
embedding_dim = 64

In [14]:

model=Sequential()

model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim))
#model.add(layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)))
model.add(layers.Dense(embedding_dim, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

2021-11-17 18:12:13.297115: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-17 18:12:13.297433: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-17 18:12:13.300437: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
dense (Dense)                (None, None, 64)          4160      
_________________________________________________________________
dense_1 (Dense)              (None, None, 6)           390       
Total params: 324,550
Trainable params: 324,550
Non-trainable params: 0
_________________________________________________________________


In [82]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs)



ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).